In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [910 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Ge

In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [13]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles

url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")


In [14]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [15]:
# Viewing the data frame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [22]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""SELECT EXTRACT(year FROM date) as Year, ROUND(AVG(price),2) as Average_Price
          FROM home_sales
          Where bedrooms == 4
          GROUP BY EXTRACT(year FROM date)""").show()


+----+-------------+
|Year|Average_Price|
+----+-------------+
|2022|    296363.88|
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [23]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""SELECT date_built, ROUND(AVG(price),2) as Average_Price
          FROM home_sales
          Where bedrooms == 3 AND bathrooms == 3
          GROUP BY date_built""").show()


+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2015|     288770.3|
|      2013|    295962.27|
|      2014|    290852.27|
|      2012|    293683.19|
|      2016|    290555.07|
|      2010|    292859.62|
|      2011|    291117.47|
|      2017|    292676.79|
+----------+-------------+



In [24]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""SELECT date_built, ROUND(AVG(price),2) as Average_Price
          FROM home_sales
          Where bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000
          GROUP BY date_built""").show()



+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2015|    297609.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2012|    307539.97|
|      2016|     293965.1|
|      2010|    285010.22|
|      2011|    276553.81|
|      2017|    280317.58|
+----------+-------------+



In [28]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT ROUND(AVG(price),2) as Average_Price, view
          FROM home_sales
          Where price >= 350000
          GROUP BY view
          ORDER BY view DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_Price|view|
+-------------+----+
|    1026669.5| 100|
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
+-------------+----+
only showing top 20 rows

--- 2.0332138538360596 seconds ---


In [30]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [31]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [32]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""SELECT ROUND(AVG(price),2) as Average_Price, view
          FROM home_sales
          Where price >= 350000
          GROUP BY view
          ORDER BY view DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|Average_Price|view|
+-------------+----+
|    1026669.5| 100|
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
+-------------+----+
only showing top 20 rows

--- 0.9174697399139404 seconds ---


In [34]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [35]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet('home_sales_partitioned')

In [36]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('parquet_home_sales')

In [37]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""SELECT ROUND(AVG(price),2) as Average_Price, view
          FROM parquet_home_sales
          Where price >= 350000
          GROUP BY view
          ORDER BY view DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|Average_Price|view|
+-------------+----+
|    1026669.5| 100|
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
+-------------+----+
only showing top 20 rows

--- 1.4589176177978516 seconds ---


In [38]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [39]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False